In [ ]:
from itertools import chain
from pathlib import Path
from random import sample

from import_analysis import ImportPatient
from skrt.registration import get_default_pfiles_dir, Registration, set_elastix_dir

# Define paths to patient data.
data_dir = Path("~/data/20220331_import_data_selection").expanduser()
paths = sorted(list(chain(data_dir.glob('import_high/H*'),
                          data_dir.glob('import_low/L*'))))

# Set path to Elastix installation.
elastix_dir = Path('~/sw/elastix-5.0.1').expanduser()
set_elastix_dir(elastix_dir)

# Obtain path to default Elastix parameter files.
pfiles_dir = get_default_pfiles_dir() 

In [ ]:
# Load data.
patients = []
for path in paths[0:1]:
    patients.append(ImportPatient(path, load_dose_sum=False, load_masks=True))
    print(f"{patients[-1].id} - initialisation; {patients[-1]._init_time:.2f} s; ")

In [ ]:
# For selected patient, create registration with planning as fixed image
idx = 0
p = patients[idx]
reg = Registration(
    Path("results/planning_fixed") / p.id,
    fixed=p.get_ct_plan(),
    moving=p.get_ct_relapse(),
    fixed_mask=p.get_mask_plan(),
    initial_alignment = p.get_centroid_translation("spinal_canal"),
    pfiles={
        #"translation": pfiles_dir / "MI_Translation.txt",
        "rigid": pfiles_dir / "MI_Rigid.txt",
        "bspline": pfiles_dir / "MI_BSpline30.txt"
    },
    overwrite=True,
    capture_output=True
)

In [ ]:
reg.register()

In [ ]:
for step in reg.steps:
    print(step)
    reg.view_result(step)